# ITK PCA-Based Transform Initialization

The SlicerMorph registration pipeline takes place in three steps:
    - Initialization: Meshes are roughly aligned to the same space
    - Rigid registration: Meshes are closely aligned without deformation via ICP
    - Deformable registration: Meshes are deformed to match with the Thin Shells Demon algorithm
    
This notebook demonstrates one way in which existing ITK filters may be leveraged to get a "close enough" mesh initialization result. Principal components are computed with an image approximation of each mesh and subsequently employed to axis-align the two meshes.

One pitfall is that PCA may fail to align the meshes if they start with opposing orientation. Moment-based initialization should be further investigated to overcome this failure.

In [1]:
import itk
import vtk
import itkwidgets

In [2]:
FIXED_MESH_FILE = r'data/129S1_SVIMJ_.ply'
MOVING_MESH_FILE = r'data/129X1_SVJ_.ply'
paths = [FIXED_MESH_FILE, MOVING_MESH_FILE]

In [3]:
import os
import importlib
from urllib.request import urlretrieve

# Download meshes
os.makedirs('data',exist_ok=True)
if not os.path.exists(FIXED_MESH_FILE):
    url = 'https://github.com/SlicerMorph/Mouse_Models/raw/main/Models/129S1_SVIMJ_.ply'
    urlretrieve(url, FIXED_MESH_FILE)
if not os.path.exists(MOVING_MESH_FILE):
    url = 'https://github.com/SlicerMorph/Mouse_Models/raw/main/Models/129X1_SVJ_.ply'
    urlretrieve(url, MOVING_MESH_FILE)

In [14]:
vtk_meshes = list()

for path in paths:
    reader = vtk.vtkPLYReader()
    reader.SetFileName(path)
    reader.Update()
    vtk_meshes.append(reader.GetOutput())
    
# Write back out to a filetype supported by ITK
vtk_paths = [path.strip('.ply') + '.obj' for path in paths]
for idx, mesh in enumerate(vtk_meshes):
    writer = vtk.vtkOBJWriter()
    writer.SetInputData(mesh)
    writer.SetFileName(vtk_paths[idx])
    writer.Update()
    
itk_meshes = [itk.meshread(path,pixel_type=itk.UC) for path in vtk_paths]

view = itkwidgets.view(geometries=[x for x in itk_meshes])

In [16]:
itk_images = [itk.triangle_mesh_to_binary_image_filter(mesh,
                                                       origin=[0,0,0],
                                                       spacing=[0.5,0.5,0.5],
                                                       size=[50,50,50])
              for mesh in itk_meshes]

In [17]:
itkwidgets.checkerboard(*itk_images)

In [18]:
itk_transforms = list()

for image in itk_images:
    calculator = itk.ImageMomentsCalculator[type(image)].New()
    calculator.SetImage(image)
    calculator.Compute()
    itk_transforms.append(calculator.GetPhysicalAxesToPrincipalAxesTransform())

In [19]:
itk_transformed_meshes = [
    itk.transform_mesh_filter(mesh, transform=itk_transforms[idx])
    for idx, mesh in enumerate(itk_meshes)
]

In [20]:
itkwidgets.view(geometries=itk_transformed_meshes)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'numberOfComponents': 3, 'd…